## Neural Networks: 

from https://pad.gwdg.de/s/Machine_Learning_For_Physicists_2021#

## Reinforcement learning


                   observation
              --------------------(ENVIRONMENT)   Fully observed vs. partially observed 'state' of the environment
              |                       ^
              v                       |
            (AGENT) -------------------
                         action

    Agent may be: self-driving car, robot -> observes inmediate environment & moves. 

- Train a net to produce actions based on rare rewards instead of being told the correct action. 

**Challenge**: the correct action is unknown -> **NO SUPERVISED LEARNING**; reward will be rare (or decided only at end). We could use the final reward to define a cost function, but we cannot know how the environment reacts to a proposed chage of the actions that were taken (UNLESS we have a model of environment). 

E.g., game

        State  = full map
        Action = move
        Reward - based on how many treasures were collected

**MODEL-FREE GENERAL REINFORCEMENT LEARNING TECHNIQUES** 

**A. REINFORCE: Policy gradient**: use probabilistic action choice. If the reward at the end turns out to be high, make **all** the actions in this sequence **more likely** (otherwise, do the opposite). This also reinforces 'bad' actions, but since they occur more likely in trajectories with low reward, the net effect will still be to suppress them. 

**B.**: